In [ ]:
# Define the number of acquisition volumes for the optimised protocol and the number TADRED iterations.
# Recall from paper we train on iteratively smaller volume sizes across a loop t = 1, ..., T
# This determines the length of the superdesign required for the training set, e.g. if n_iterations_tadred
# is 5 then the superdesign needs to be 16 times larger than the number of volumes in the optimised protocol

# TODO doesn't it make more sense to put this where we select the subsets? unless if this is the main part we want the user to vary

n_volumes_opt_protocol = 20
n_iterations_tadred = 5 # T in paper

print(f"Length of desired optimised protocol: {n_volumes_opt_protocol}")
print(f"Number of TADRED iterations: {n_iterations_tadred}")


In [ ]:
# Calculate the required length of the superdesign, this depends on the number of 
# this depends on the number of TADRED iterations - which are always halvings - that will be done in tadred
Vbar = n_volumes_opt_protocol * 2**(n_iterations_tadred - 1) #

print("Size of the required superdesign for the simulations: {Vbar}")


In [ ]:
########## (1)
# See requirements.txt for tadred requirements, make sure things are in the path, set global seed

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from tadred import tadred_main, utils

# Replace below with directory data already is, or whre you want to save the data and results 
out_base: str = '/home/blumberg/Bureau/z_Automated_Measurement/Output/journal_paper_tst' # ''
Path(out_base).mkdir(parents=True, exist_ok=True)

# If TADRED is installed in another directory, add the location tobelow
# TADRED_dir: str = ''
# sys.path.append(TADRED_dir)

np.random.seed(0)  # Random seed for entire script


In [ ]:
########## (2)
# Data split sizes

n_train_voxels = 10**4  # Reduce for faster training speed
n_val_voxels = n_train_voxels // 10
n_test_voxels = n_train_voxels // 10
n_samples = n_train_voxels + n_val_voxels + n_test_voxels  # total number of samples to simulate

use_own_simulated_data: bool = False


In [ ]:
#If you are using your own simulated data, follow the guidelines to 
#generate simulated data that is appropriate for input to TADRED

# TODO this should be in the introduction where we define what the data looks like.  Or in a different repo.

if use_own_simulated_data:
    print(
        f"""
        You need to simulate the following data to run TADRED
        Ground truth parameters vector of size n_samples by n_model_parameters
        Where:
            n_samples is {n_samples}
            n_model_parameters is the number of parameters in your model
        
        Simulated ground truth signals with superdesign acquisition scheme of size n_samples by Vbar
        Where:
            The superdesign acquisition scheme highly oversamples the available acquisition parameter space
            Vbar is the superdesign length {Vbar}
        
        Acquisition parameters of the superdesign of size Vbar by n_acquisition_parameters
        Where:
            n_acqusition_parameters is the dimension of the acquisition parameter space, e.g. 4 (gx, gy, gz, b) for HCP data
        
        Save the data as .npy files.
        Suggested filenames:
            parameter array: parameters_gt_full.npy
            simulated signals: signals_super_full.npy
            acquisition parameters: acq_params_super.npy
    """
    )
else:
    print(f'Run the cell below to simulate data suitable for TADRED')


In [ ]:
# Define some models and generate data
# Data that will be generated is:
#   parameters - n_samples by n_model_parameters  array containing the ground truth model parameters, where n_parameters is the number of parameters in your model
#   signals - n_samples by Vbar array containing the corresponding simulated signals from the model 
#   acq_params_super - Vbar by n_acqusition_parameters length array containing the acquisition parameters of the superdesign

model_name = 't1inv' # T1 inversion recovery model
# model_name = "adc" # ADC model
SNR: int = 20
proj_name = f"{model_name}_simulations_n_train_voxels_{n_train_voxels}_SNR_{SNR}"
proj_dir = Path(out_base, proj_name)
Path.mkdir(proj_dir, exist_ok=True, parents=True)

if not use_own_simulated_data:
        
    if model_name == "adc":
        def model(D, bvals):
            signals = np.exp(-bvals * D)
            return signals
    
        # min/max parameter values
        minD = 0.1
        maxD = 3
    
        # Simulate parameter values
        parameters = np.random.uniform(low=minD, high=maxD, size=(n_samples, 1))
    
        ### Generate data using the model
    
        # Make super design
        maxb = 5
        minb = 0
        acq_params_super = np.linspace(minb, maxb, Vbar)
    
        # Generate data
        raw_signals = np.zeros((n_samples, Vbar), dtype=np.float32)
        for i in range(0, n_samples):
            raw_signals[i, :] = model(parameters[i], acq_params_super)
    
    elif model_name == "t1inv":
    
        def model(T1, ti, tr):
            signals = abs(1 - (2 * np.exp(-ti / T1)) + np.exp(-tr / T1))
            return signals
    
        # min/max parameter values
        minT1 = 0.1
        maxT1 = 7
        # Simulate parameter values
        parameters = np.random.uniform(low=minT1, high=maxT1, size=(n_samples, 1))
        
        # Make the super design
        tr = 7  # repetition time
        maxti = tr
        minti = 0.1
        acq_params_super = np.linspace(minti, maxti, Vbar)
    
        # Generate the data
        raw_signals = np.zeros((n_samples, Vbar), dtype=np.float32)
        for i in range(0, n_samples):
            raw_signals[i, :] = model(parameters[i], acq_params_super, tr)

    # Add Rician noise to the data
    def add_noise(data, scale=0.05):
        data_real = data + np.random.normal(scale=scale, size=np.shape(data))
        data_imag = np.random.normal(scale=scale, size=np.shape(data))
        data_noisy = np.sqrt(data_real**2 + data_imag**2)
    
        return data_noisy
    
    signals_super = add_noise(raw_signals, 1 / SNR)
    

In [ ]:
# TODO this cell doesn't make any sense, what are you meant to be loadikng and why?
# TODO maybe keep this notebook for simulations only

if use_own_simulated_data:
    # Replace with project name
    proj_name: str = 'TADRED_test'

    # REPLACE WITH PATH TO SIMULATION GROUND TRUTH PARAMETERS
    # Saved array should be array n_samples by n_model_parameters     
    simulation_gt_parameters_path = Path(proj_name,'parameters_gt_full.npy')
    
    # REPLACE WITH PATH TO SIMULATION GROUND TRUTH SIGNALS WITH "SUPER-DESIGN" ACQUISITION - HIGHLY OVERSAMPLING THE ACQUISITION PARAMETER SPACE
    # Array is n_samples by V_bar
    simulation_gt_signals_path = Path(proj_name, 'signals_super_full.npy')
    
    # REPLACE WITH PATH TO SUPER-DESIGN ACQUISITION PARAMETERS
    # acq_params_super - Vbar by n_acqusition_parameters length array containing the acquisition parameters of the superdesign

    
    # Array is Vbar by n_acqusition_parameters, e.g. n_acqusition_parameters is 4 (gx, gy, gz, b) for HCP data
    acq_params_super_signals_path = Path(proj_name, 'acq_params_super.npy')
    
    # Load the files
    parameters = np.load(simulation_gt_parameters_path)
    signals_super = np.load(simulation_gt_signals_path)
    acq_params_super = np.load(acq_params_super_signals_path)
    

In [ ]:
########## (3-A)
# Create dummy, randomly generated (positive) data

# C_bar = 220
# M = 12  # Number of input measurements \bar{C}, Target regressors
# rand = np.random.lognormal  # Random genenerates positive
# train_inp, train_tar = rand(size=(n_train_voxels, C_bar)), rand(size=(n_train_voxels, M))
# val_inp, val_tar = rand(size=(n_val, C_bar)), rand(size=(n_val, M))
# test_inp, test_tar = rand(size=(n_test_voxels, C_bar)), rand(size=(n_test_voxels, M))


# #########

In [ ]:
########## (4)
# Load data into TADRED format

print(signals_super)

# Data in TADRED format, \bar{V} volumes, M target regresors
data = dict(
    train=signals_super[0:n_train_voxels, :],  # Shape n_train_voxels x \bar{C}
    train_tar=parameters[0:n_train_voxels, :],  # Shape n_train_voxels x M
    val=signals_super[n_train_voxels : (n_train_voxels + n_val_voxels), :],  # Shape n_val_voxels x \bar{C}
    val_tar=parameters[n_train_voxels : (n_train_voxels + n_val_voxels), :],  # Shape n_val_voxels x M
    test=signals_super[(n_train_voxels + n_val_voxels): , :],  # Shape n_test_voxels x \bar{C}
    test_tar=parameters[(n_train_voxels + n_val_voxels):, :],  # Shape n_test_voxels x M
)

for key, value in data.items():
    data[key] = value.astype(np.float32)

# Save data to disk -- optional
np.save(Path(proj_dir, 'data'), data)    


In [ ]:
########## (6)
# Simplest version of TADRED, modifying the most important hyperparameters
# The decreasing subset sizes are hard-coded so final optimized protocol is 1/16 the size of the superdesign
# Feel free to play around with the subset size reduction pattern, 
# halving the size of the subset sizes at each TADRED step seems to generally work well.

args = utils.load_base_args()

args.tadred_train_eval.feature_set_sizes_Ci = [Vbar // (2**i) for i in range(n_iterations_tadred)]
args.tadred_train_eval.feature_set_sizes_evaluated = [Vbar // (2**i) for i in range(1, n_iterations_tadred)]

# Scoring net Vbar -> num_units_score[0] -> num_units_score[1] ... -> Vbar units
args.network.num_units_score: list[int] = [1000, 1000]

# Task net Vbar -> num_units_task[0] -> num_units_task[1] ... -> M units
args.network.num_units_task: list[int] = [1000, 1000]

# SBB TODO add Pathlib option to tadred.utils.py:create_out_dirs
args.output.out_base = str(out_base)  # "/Users/paddyslator/python/ED_MRI/test1" #"/home/blumberg/Bureau/z_Automated_Measurement/Output/paddy"
args.output.proj_name = str(proj_name)
args.output.run_name = "test"
args.other_options.save_output = True

#print(args)

args.tadred_train_eval.epochs = 50

print(out_base, proj_name)

TADRED_output = tadred_main.run(args, data)


In [ ]:
# We can also load the saved results here
# TADRED_output = np.load(
#     Path(
#         proj_dir,
#         "results",
#         args.output.run_name + "_all.npy",
#     ),
#     allow_pickle=True,
# ).item()


In [ ]:
# Extract some useful parameters fom the tadred output
# final subset index
V_last = TADRED_output["args"]["tadred_train_eval"]["feature_set_sizes_Ci"][-1] # V_{T} in paper

# Index of chosen acquisition parameters
acq_params_tadred_index = TADRED_output[V_last]["measurements"]

# Chosen acquisition parameters
acq_params_tadred = acq_params_super[acq_params_tadred_index]

print(f"TADRED chosen acquisition parameters are: {acq_params_tadred}")

In [ ]:
#plot the signals at the super design and the tadred chosen for a single voxel
#

#if the number of acquisition parameters is bigger than one, need to choose which one to plot on the x axis
if (acq_params_super.ndim > 1): 
    if (acq_params_super.shape[1] > 1):
        acq_param_to_plot = 1
        acq_params_super_to_plot = acq_params_super[:,acq_param_to_plot]
        acq_params_tadred_to_plot = acq_params_tadred[:,acq_param_to_plot]
        acq_params_tadred_index_to_plot = acq_params_tadred_index[:,acq_param_to_plot]
else:
    acq_params_super_to_plot = acq_params_super
    acq_params_tadred_to_plot = acq_params_tadred
    acq_params_tadred_index_to_plot = acq_params_tadred_index
    
voxel_to_plot = 0


plt.plot(acq_params_super_to_plot, signals_super[voxel_to_plot,:], 'x')
plt.plot(acq_params_tadred_to_plot, signals_super[voxel_to_plot,acq_params_tadred_index], 'o')

plt.title('signals from voxel ' + str(voxel_to_plot))
plt.legend(('Super design', 'TADRED chosen'))
plt.ylabel('signal')
plt.xlabel('acquisition parameter')

In [ ]:
# save TADRED acquisition parameters
#np.save(Path(proj_, "acq_params_tadred.npy"), acq_params_tadred)